In [2]:

import os
!pip install nltk matplotlib
!pip install bs4 scipy tabulate glob dask textstat json glob distributed
!pip install textstat
!pip install tabulate textblob

#import matplotlib
#%matplotlib inline



The directory '/Users/rjjarvis/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/rjjarvis/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/rjjarvis/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/rjjarvis/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo,

In [ ]:
#general python imports
import os
#import dask
import matplotlib # Its not that this file is responsible for doing plotting, but it calls many modules that are, such that it needs to pre-empt
# setting of an appropriate backend.
#matplotlib.use('Agg')
import sys
import numpy
import numpy as np
import scipy
import scipy.io as sio
import math
import re
#import requests
import time
from tabulate import tabulate
from textblob import TextBlob
import glob

#text analysis imports
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tag.perceptron import PerceptronTagger
tagger = PerceptronTagger(load=False)

from nltk import word_tokenize,sent_tokenize
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import cmudict
from nltk.sentiment import SentimentAnalyzer
from nltk import compat
#from nltk.compat import Counter
#from nltk.draw import dispersion_plot

from bs4 import BeautifulSoup
import json

from textstat.textstat import textstat

########################################################################
########################################################################
########################################################################
# Set user parameters based on type of analysis
searchList = ['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']
nweb = 2 #number of search websites being implemented (google, google scholar, bing, yahoo)
numURLs = 10 #number of URLs per search website  (number determined by 1.scrape code)

#also save these parameters for analysis purposes
spec_dict = { 'searchList':searchList, 'nweb':nweb, 'numURLs':numURLs}
handle = 'Data/analysisSpecs.mat'
scipy.io.savemat(handle, mdict=spec_dict, oned_as='row')

########################################################################
########################################################################
########################################################################
#what are we analyzing???? - this is the list of text analysis features
infoDat = {}
infoDat[1] = "Number of Words"
infoDat[2] = "Number of Sentences"
infoDat[3] = "Frequency of Search Term"
infoDat[4] = "Sentiment Analysis"
infoDat[5] = "Subjectivity Analysis"
infoDat[6] = "Grade level"
infoDat[7] = "Flesch Reading Ease"
infoDat[8] = "SMOG Index"
infoDat[9] = "Coleman Liau"
infoDat[10] = "Automated Readability Index"
infoDat[11] = "Gunning Fog"
infoDat[12] = "Dale Chall Readability Score"
infoDat[13] = "Difficult Words"
infoDat[14] = "Linsear Write Formula"
infoDat[15] = "Text Standard"

#save these parameters for analysis purposes
info_dict = { 'infoDat':infoDat}
handle = 'Data/analysisInfo.mat'
scipy.io.savemat(handle, mdict=info_dict, oned_as='row')

########################################################################
########################################################################
########################################################################
#set filePath below to specify where the data will be going after the code runs
fileLocation = os.getcwd()

if not os.path.exists(fileLocation):
    os.makedirs(fileLocation)

########################################################################
########################################################################
########################################################################
#start all the analysis code

#for s, value in enumerate(searchList):
#def iter_over(searchListElement):
sl = [ (i, val) for i, val in enumerate(searchList) ]
for s, value in sl:
    #s, value = searchListElement

    if not os.path.exists(str(fileLocation) + '/' + str(value) +'/'):
        os.makedirs(str(fileLocation) + '/' + str(value) +'/')
    os.chdir(fileLocation +str('/') + str(value) +'/')

    print (" ")
    print ("###############################################")
    print (" ")
    print ("Term {0} of {1} : {2}".format(s+1 , str(len(searchList)), value))
    print (" ")
    print ("###############################################")
    #web = [ "google_","gScholar_","bing_","yahoo_" ]
    #web = [0:nweb]

    for b in range(0,nweb):
    #for b, _ in enumerate(web):
        #search engine selection
        if b == 0:
            textName = "google_"
            print ("Google")
        elif b == 1:
            textName = "gScholar_"
            print ("Google Scholar")
        elif b == 2:
            textName = "bing_"
            print ("Bing")
        elif b == 3:
            textName = "yahoo_"
            print ("Yahoo")

        #import pdb; pdb.set_trace()
        #list_of_files = glob.glob(r'textName*.p')
        list_of_files = sorted(glob.glob(str(textName)+r'*.p'))
        #if len(list_of_files) >= 50:
        list_of_files =  sorted(list_of_files[0:numURLs-1])

        for p,fileName in enumerate(list_of_files):
            print ("-------------------------------------------")
            print ("Analyzing Search Engine " + str(b+1) + " of " + str(nweb) + ": Link " + str(p)); print ("");
            #open and read text q

            #fileName = l#'{0}{1}.p'.format(textName,p+1)
            print(fileName)
            print(os.getcwd(), 'pwd')
            import pickle

            fileHandle = open(fileName, 'rb');
            file_contents = pickle.load(fileHandle)
            if len(file_contents) == 2:
                date_created = file_contents[0]
                url_text = file_contents[1]
            else:

                url_text = file_contents

            #initialize dataArray Dictionary
            urlDat = {}

            ########################################################################
            #remove unreadable characters
            url_text = url_text.replace("-", " ") #remove characters that nltk can't read
            textNum = re.findall(r'\d', url_text) #locate numbers that nltk cannot see to analyze
            for x in range(0,len(textNum)) :
                url_text.find(textNum[x])

            ########################################################################
            ##Splitting text into:
            #words.
            URLtext = word_tokenize(url_text)
            URLtext = [w.lower() for w in URLtext] #make everything lower case

            urlDat[1] = textstat.lexicon_count(str(url_text))
            #import pdb; pdb.set_trace()

            #sentences
            sents = sent_tokenize(url_text) #split all of text in to sentences
            sents = [w.lower() for w in sents] #lowercase all

            urlDat[2] = len(sents) #determine number of sentences

            ########################################################################
            ##frequency distribtuion of text
            fdist = FreqDist(w.lower() for w in URLtext if w.isalpha()) #frequency distribution of words only

            # Bug fix
            # cast dict to list
            fd_temp = list(fdist.items())

            urlDat[3] = fdist[searchList[s].lower()] #frequency of search term

            #frequency of all words
            fAll = {}
            for x in range(0,len(fd_temp)):
                fAll[x,1], fAll[x,2] = [y.strip('}()",{:') for y in (str(fd_temp[x])).split(',')]

            #frequency of the most used n number of words
            frexMost = fdist.most_common(15) #show N most common words
            fM = {}
            for x in range(0,len(frexMost)) :
                fM[x,1], fM[x,2] = [y.strip('}()",{:') for y in (str(frexMost[x])).split(',')]

            ########################################################################
            #Sentiment and Subjectivity analysis
            testimonial = TextBlob(url_text)
            testimonial.sentiment

            urlDat[4] = testimonial.sentiment.polarity
            urlDat[5] = testimonial.sentiment.subjectivity

            ########################################################################
            #determine syllable count for all words in each sentece
            sentSyl = {}
            WperS = {}
            # for n,sent in enumerate(sents):
            # future use
            for n in range(0,len(sents)):

                #setup sent variable to analyze each sentence individually
                sent = sents[n] #select sentence n in total text
                sent = word_tokenize(sent) #tokenize sentence n in to words
                sent = [w.lower() for w in sent if w.isalpha()] #remove any non-text

                WperS[n] = len(sent) #number of words per sentence

                #syllable analysis

                for x in range(0,len(sent)):
                    word = sent[x]

                    # Count the syllables in the word.
                    syllables = textstat.syllable_count(str(word))
                    sentSyl[n,x] = syllables
            #
            ########################################################################
            ## Complexity Analysis
            #try:
            assert len(url_text) != 0
            assert type(url_text) is not type(None)
            urlDat[6]  = textstat.flesch_kincaid_grade(str(url_text))
            urlDat[7] = textstat.flesch_reading_ease(str(url_text))
            urlDat[8]  = textstat.smog_index(str(url_text))
            urlDat[9]  = textstat.coleman_liau_index(str(url_text))
            urlDat[10]  = textstat.automated_readability_index(str(url_text))
            urlDat[11] = textstat.gunning_fog(str(url_text))

            urlDat[12]  = textstat.dale_chall_readability_score(str(url_text))
            urlDat[13]  = textstat.difficult_words(str(url_text))
            urlDat[14]  = textstat.linsear_write_formula(str(url_text))
            urlDat[15]  = textstat.text_standard(str(url_text))
            #import pdb
            #pdb.set_trace()

            ########################################################################
            ########################################################################
            ########################################################################
            ########################################################################
            #clean-up and prep for saving for subsequent analysis and plotting

            ##convert all dict variables to list for multidimensional conversion to matlab cell array
            urlDat = list(urlDat.items())
            sentSyl = list(sentSyl.items())
            WperS = list(WperS.items())
            fAll = list(fAll.items())
            fM = list(fM.items())

            ##generate a .mat file for further analysis in matlab
            if b == 0 and p == 0:
                obj_arr = [urlDat, WperS, sentSyl, fM, fAll]
                #obj_arr = np.array([urlDat, WperS, sentSyl, fM, fAll], dtype=np.object)
                print('dimensions change of object array: ',np.shape(obj_arr),np.shape(urlDat))
                old = np.shape(obj_arr)

            else:
                obj_arr_add = [urlDat, WperS, sentSyl, fM, fAll]
                #obj_arr_add = np.array([urlDat, WperS, sentSyl, fM, fAll], dtype=np.object)

                print('dimensions change of object array: ',np.shape(obj_arr),np.shape(urlDat))

                obj_arr = np.vstack( [obj_arr, obj_arr_add])
                assert type(obj_arr) is not type(None)
                assert np.shape(obj_arr) != old
                old = np.shape(obj_arr)

            handle = str('../Data/') + str(searchList[s]) + str('_term_')+ str(s)+ '.mat'
            import scipy
            scipy.io.savemat(handle, {'obj_arr':obj_arr} , oned_as='row')

In [2]:
sl = [ (i, val) for i, val in enumerate(t_analysis.searchList) ]
import dask.bag as db
b = db.from_sequence(sl, npartitions=8)
print(b)
print(type(b))

##
#
#

obj_arrs = list(db.map(t_analysis.iter_over,b).compute())#.result()
import pandas as pd


dask.bag<from_se..., npartitions=1>
<class 'dask.bag.core.Bag'>
 
###############################################
 
Term 1 of 1 : GMO
 
###############################################
Google
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 0

google_0.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 1

google_1.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 2

google_10.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_'


google_4.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 35

google_40.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 36

google_41.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 37

google_42.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 1 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 38

google_43.p
/home/jovyan/SReadability/GMO pwd
number of words is zer

/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 2 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 21

gScholar_28.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 2 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 22

gScholar_29.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 2 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 23

gScholar_3.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 2 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 24

gScholar_30.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on 

number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 3 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 7

bing_15.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 3 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 8

bing_16.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 3 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 9

bing_17.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------------------------------
Analyzing Search Engine 3 of ['google_', 'gScholar_', 'bing_', 'yahoo_']: Link 10

bing_18.p
/home/jovyan/SReadability/GMO pwd
number of words is zero on that link, so analysis will fail
-------------------

In [3]:
df = pd.DataFrame(data=obj_arrs)
df

,0
0,None


In [ ]:
#df2 = df[0][1]
#df.iloc[3]

In [ ]:
#df2

In [ ]:
len(df.loc[0])

%matplotlib inline
import matplotlib.pyplot as plt
#import matplotlib as plt
plt.figure();
df.iloc[0][3]
#plt.plot(,kind='hist');
'''
for i in df.iloc[0][3]:
    print(type(i[0]),type(i[1]))
    print(i[0],i[1])#,i[2])
'''    
    #print(type(i[1]))
#df.loc[0][3]
#df.loc[0][4]
#df.loc[0][5]
#df.loc[0][6]
len(obj_arrs[3])
obj_arrs[0][3]
obj_arrs[0][3]


In [ ]:
TermName = 'GMO'

numVars = 13;
word = len(TermName);

web = 1#; %how many search engines
numPerURL = 1#; %how many URLS per search engine
crawlCount = 5#; %how many links are being crawled per URL


#for wordi = 1:word
for wordi in TermName:
    print(wordi)
    

metricAvg = []
metricAvg.append(8)#; %grade level
metricAvg.append(65)#; %FK reading ease
metricAvg.append(8)#; %FK grade level
metricAvg.append(8)#; %Gunning Fog
metricAvg.append(25.5)#; %SMOG
metricAvg.append(8)#; %Coleman
metricAvg.append(9)#; %Automated Readability Index
'''
    for webi = 1:web
        for n = 1:7
            %find min and max values for each metric
            tempMin = min([plotData{webi,2}(n,:) metricAvg(n,1)]);
            tempMax = max([plotData{webi,2}(n,:) metricAvg(n,1)]);

            %normalize
            for i = 1:numPerURL
                NplotData{webi,2}(n,i) = (plotData{webi,2}(n,i) - tempMin)/(tempMax-tempMin);
                metricAvg(n,2) = (metricAvg(n,1) - tempMin)/(tempMax-tempMin);
            end; clear tempMin tempMax
        end
    end; clear webi n i

    %save data for each keyword for later plot comparison
    dataFinal{wordi,1} = DataFinal; clear DataFinal
    MetricAvg{wordi,1} = metricAvg; clear metricAvg
    PlotData{wordi,1} = plotData; clear plotData
    normPlotData{wordi,1} = NplotData; clear NplotData
'''